In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module meshsplit
loading module palettecontrols
loading module volumepalettes


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-jc225751'


loading module save_resampled
loading module profilewindow
loading module statsplotwindow
loading module anacontrolmenu
loading module infowindow
loading module foldsplit
loading module simple_controls
loading module histogram
loading module valuesplotwindow
loading module ana_image_math
loading module modelGraphs
loading module paletteViewer
loading module bsa_proba
loading module gradientpalette
loading module selection
loading module gltf_io
all python modules loaded
Anatomist started.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import umap
import os

### In this example, the embeddings are generated from a Barlow Twin model trained on the Left CINGULATE region on UKB for the UKB and HCP datasets.

 -For the HCP dataset, the associated buckets can be found in: 

/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Lbuckets

 -For the UKB dataset, the buckets are not generated yet, but it would be found in:

/neurospin/dico/data/deep_folding/current/datasets/UkBioBank/crops/2mm/CINGULATE/mask/Lbuckets

### Load your data

In [3]:
embeddings_UKB = pd.read_csv('/neurospin/dico/data/deep_folding/current/models/Champollion_V0/SC-sylv_right/11-43-38_3/ukb40_random_epoch100_embeddings/full_embeddings.csv', index_col=0)
labelled_CS = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/CSLabel/interrupted_CS_labelled.csv')

list_problem = [
'sub-3716267',
'sub-5417598',
'sub-5472164',
'sub-5894417',
'sub-1703355',
'sub-4271898',
]

list_interrupted_CS = labelled_CS[labelled_CS.Interruption==1].ID.to_list()
list_interrupted = ['0' if i not in list_interrupted_CS else '1' for i in embeddings_UKB.index]
list_color = ['g' if i not in list_interrupted_CS else 'r' for i in embeddings_UKB.index]
list_size = [0.3 if i not in list_interrupted_CS else 30 for i in embeddings_UKB.index]
list_label = ['0' if i not in list_interrupted_CS else '1' for i in embeddings_UKB.index]
embeddings_UKB.head()

,dim1,dim2,dim3,dim4,dim5,dim6,dim7,dim8,dim9,dim10,...,dim247,dim248,dim249,dim250,dim251,dim252,dim253,dim254,dim255,dim256
ID,,,,,,,,,,,,,,,,,,,,,
sub-1000021,-37.629620,6.583590,29.279840,-16.611984,-30.081190,-24.476210,11.042302,-6.784241,19.132568,10.150197,...,-2.848078,11.085325,-12.354747,-76.037210,-8.468971,-29.602938,-24.906027,-16.344492,7.703534,-1.050617
sub-1000325,11.934219,11.355382,-13.496794,-38.996407,14.297873,-5.811631,6.174108,-45.331050,9.762536,44.107750,...,4.212949,6.075599,-36.018467,-134.833560,26.447592,-35.490543,4.641970,-4.289551,-58.478560,-31.515226
sub-1000458,-25.394463,4.755465,-22.513622,-25.682335,-67.619896,-18.027046,-30.610876,51.657852,-18.840885,-7.459018,...,-9.459454,-0.912658,32.695583,12.509913,10.790437,-25.991050,-39.407890,-13.558734,47.675068,-0.354762
sub-1000575,-25.367662,-23.005732,-30.035063,36.237038,-103.579190,-9.643719,4.125421,34.603440,19.479185,-13.243917,...,20.220034,-5.049628,4.961305,-25.468930,25.991018,3.052118,-73.673270,6.855866,30.889896,43.402153
sub-1000606,-35.599197,-2.058082,26.222765,-3.495081,-49.495464,-3.061627,-14.236676,-3.899254,-15.054834,50.399190,...,8.652889,27.369568,34.453293,-73.026400,40.409150,28.420736,-51.241203,17.027500,-0.707319,-20.734170


### 2D UMAP

In [4]:
reducer2D = umap.UMAP(densmap=True, dens_lambda=10)
reducer2D.fit(embeddings_UKB)
bdd_2D_UKB = reducer2D.transform(embeddings_UKB)

/casa/home/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [5]:
bdd_2D_UKB = pd.DataFrame(bdd_2D_UKB, columns=['Dim 1', 'Dim 2'])
bdd_2D_UKB['Dataset'] = 'UkBioBank'
bdd_2D_UKB['ID'] = embeddings_UKB.index
bdd_2D_UKB['Interruption'] = list_interrupted
bdd_2D_UKB.head()

,Dim 1,Dim 2,Dataset,ID,Interruption
0,5.577102,3.288216,UkBioBank,sub-1000021,0
1,2.404181,3.121142,UkBioBank,sub-1000325,0
2,8.570886,6.044249,UkBioBank,sub-1000458,0
3,10.377075,5.906775,UkBioBank,sub-1000575,0
4,9.827856,2.111227,UkBioBank,sub-1000606,0


### Plot it in the notebook or write a html file

In [6]:
subject_id_list = []
dataset_name_list = []

In [7]:
# Create the scatter plot using plotly express
fig = px.scatter(
    bdd_2D_UKB, x='Dim 1', y='Dim 2', 
    color='Dataset',
    title='CS interruption UKB',
    labels={'0': 'dim 1', '1': 'dim 2'},
    hover_data= ['Dataset', 'ID'],
    opacity=0.5,
    width=800, height=600
)

# Convert the figure to a FigureWidget
f = go.FigureWidget(fig)

# Define the callback function
def click_callback(trace, points, selector):
    for trace_index in range(len(f.data)):
        if trace_index == points.trace_index:
            customdata = f.data[trace_index].customdata
            for i in points.point_inds:
                point_dataset, point_id = customdata[i]
                print(f"Clicked point ID: {point_id}, Dataset: {point_dataset}")
                subject_id_list.append(point_id)
                dataset_name_list.append(point_dataset)

# Attach the callback to the on_click event for all traces
for trace in f.data:
    trace.on_click(click_callback)

# Display the figure widget
f

FigureWidget({
    'data': [{'customdata': array([['UkBioBank', 'sub-1000021'],
                                   ['UkBioBank', 'sub-1000325'],
                                   ['UkBioBank', 'sub-1000458'],
                                   ...,
                                   ['UkBioBank', 'sub-6024150'],
                                   ['UkBioBank', 'sub-6024379'],
                                   ['UkBioBank', 'sub-6024754']], dtype=object),
              'hovertemplate': ('Dataset=%{customdata[0]}<br>Di' ... '{customdata[1]}<extra></extra>'),
              'legendgroup': 'UkBioBank',
              'marker': {'color': '#636efa', 'opacity': 0.5, 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'UkBioBank',
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'f40c93ce-7961-44fc-ada0-1ce6864e8a29',
              'x': array([5.577102 , 2.4041815, 8.570886 , ..., 5.672671 , 4.804484 , 7.8271613],
               

Clicked point ID: sub-5053872, Dataset: UkBioBank
Clicked point ID: sub-5540476, Dataset: UkBioBank
Clicked point ID: sub-5626543, Dataset: UkBioBank
Clicked point ID: sub-3416453, Dataset: UkBioBank
Clicked point ID: sub-4760288, Dataset: UkBioBank
Clicked point ID: sub-4381871, Dataset: UkBioBank
Clicked point ID: sub-4484599, Dataset: UkBioBank
Exiting QApplication


In [8]:
dataset = 'UkBioBank40'
region = "S.C.-sylv."
side = "R"

mm_skeleton_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'

In [9]:
volume_files = []

for subject_id in set(subject_id_list):
    volume_path = f"{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz"
    
    if os.path.isfile(volume_path):
        vol = aims.read(volume_path)
        volume_files.append(vol)
    else:
        print(f"{volume_path} is not a correct path, or the .nii.gz doesn't exist")

block = a.createWindowsBlock(4) # number of columns
dic_windows = {}

for i, vol in enumerate(volume_files):
    dic_windows[f'a_vol{i}'] = a.toAObject(vol)
    #dic_windows[f'a_vol{i}'].setPalette(absoluteMode=True)
    dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
    dic_windows[f'rvol{i}'].releaseAppRef()
    dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block) #geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
    dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])

Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


: 